# Analisis de duracion de partidos de tenis
## Estadistica aplicada 2
### Integrantes
- Juan Pablo Cordero
- Jaime Uria
- Gerardo Guerrero

# Importamos las librerías necesarias


In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 1) Load the CSV
matches = pd.read_csv("atp_data/atp_matches_till_2022.csv")


# Vemos los valores faltantes que se ven en el DF

In [72]:
# 2) Inspect and clean
print(matches['minutes'].describe())      # look at distribution & missing
print(matches['minutes'].isna().sum())  # check for missing values
nan_percentage = matches['minutes'].isna().mean() * 100
print(f"Percentage of NaN values in 'minutes': {nan_percentage:.2f}%")  # check for missing values
# Method 1: Get total length of the DataFrame or column
total_count = len(matches['minutes'])
print(f"Total records (including NaN): {total_count}")


count    89511.000000
mean       103.497403
std         39.365772
min          0.000000
25%         75.000000
50%         96.000000
75%        125.000000
max       1146.000000
Name: minutes, dtype: float64
98650
Percentage of NaN values in 'minutes': 52.43%
Total records (including NaN): 188161


# Tiramos los partidos que no tienen duracion

In [73]:
df = matches.dropna(subset=['minutes'])   # drop matches without a duration

# Variables del Dataset de Tenis - Descripción

| Variable | Descripción | Tipo | Valores |
|----------|-------------|------|---------|
| `tourney_id` | Identificador único del torneo | String | Alfanumérico |
| `tourney_name` | Nombre del torneo | String | Texto |
| `surface` | Superficie en la que se juega el partido | Categórica | Grass, Carpet, Clay, Hard |
| `draw_size` | Tamaño del cuadro del torneo | Numérico | Entero (32, 64, 128, etc.) |
| `tourney_level` | Nivel del torneo | Categórica | G = Grand Slams<br>M = Masters 1000<br>A = Otros eventos ATP Tour<br>C = Challengers<br>S = Satellites/ITFs<br>F = Finales de temporada<br>D = Copa Davis |
| `tourney_date` | Fecha de inicio del torneo | Fecha | YYYYMMDD |
| `match_num` | Número de partido en un torneo específico | Numérico | Entero |
| `id` | Identificador del jugador | String | Alfanumérico |
| `seed` | Cabeza de serie del jugador en el torneo | Numérico | Entero o NaN |
| `entry` | Forma de entrada al torneo | Categórica | WC = Wildcard<br>Q = Clasificado<br>LL = Lucky loser<br>PR = Protected ranking<br>SE = Special Exempt<br>ALT = Alternate player |
| `name` | Nombre del jugador | String | Texto |
| `hand` | Mano dominante del jugador | Categórica | R = Diestro<br>L = Zurdo |
| `ht` | Altura del jugador | Numérico | Entero (en cm) |
| `IOC` | País de origen (código olímpico) | Categórica | Códigos de 3 letras |
| `age` | Edad del jugador | Numérico | Decimal |
| `score` | Resultado final del partido | String | Texto formateado |
| `best_of` | Número máximo de sets en el partido | Numérico | 3 o 5 |
| `round` | Ronda del torneo | Categórica | R128 = Ronda de 128<br>R64 = Ronda de 64<br>R32 = Ronda de 32<br>R16 = Octavos de final<br>QF = Cuartos de final<br>SF = Semifinales<br>F = Final<br>RR = Round Robin<br>ER = Early Round<br>BR = Bronze medal match |
| `minutes` | Duración del partido en minutos | Numérico | Entero |
| `ace` | Número de aces en el partido | Numérico | Entero |
| `df` | Dobles faltas | Numérico | Entero |
| `svpt` | Puntos de servicio | Numérico | Entero |
| `1stin` | Primer servicio dentro (cantidad) | Numérico | Entero |
| `1stWon` | Puntos ganados con primer servicio | Numérico | Entero |
| `2ndWon` | Puntos ganados con segundo servicio | Numérico | Entero |
| `SvGms` | Número de juegos al servicio | Numérico | Entero |
| `bpSaved` | Puntos de break salvados | Numérico | Entero |
| `bpFaced` | Puntos de break enfrentados | Numérico | Entero |

## Notas Adicionales:

- Para las estadísticas de partido (ace, df, svpt, etc.), hay dos versiones de cada variable:
  - Prefijo `w_` para el ganador del partido (winner)
  - Prefijo `l_` para el perdedor del partido (loser)

# Nota:
### Solo nos vamos a centrar en los torneos Grand Slams y Masters 1000

In [74]:
t_lev = ["G", "M"]

df = df[df['tourney_level'].isin(t_lev)]  # filter for Grand Slam and Masters tournaments
display(df['tourney_level'].value_counts())  # check the distribution of tournament levels

tourney_level
M    16915
G    14254
Name: count, dtype: int64

### Confirmamos que los torneos son Grand Slams y Masters 1000

In [75]:
# convert all tournament names to lowercase and strip whitespace
df['tourney_name'] = df['tourney_name'].str.lower().str.strip()

# verify
sorted(df['tourney_name'].unique())


['australian open',
 'canada masters',
 'cincinnati masters',
 'essen masters',
 'hamburg masters',
 'indian wells masters',
 'madrid masters',
 'miami masters',
 'monte carlo masters',
 'paris masters',
 'roland garros',
 'rome masters',
 'shanghai masters',
 'stockholm masters',
 'stuttgart masters',
 'us open',
 'wimbledon']

### Poner la fecha en formato correcto

In [76]:
# if tourney_date is int, first convert to string
df['tourney_date'] = pd.to_datetime(
    df['tourney_date'].astype(str),
    format='%Y%m%d'
)
# now verify
print(df['tourney_date'].dtype)
print(df['tourney_date'].head())


datetime64[ns]
85972   1991-01-14
85973   1991-01-14
85974   1991-01-14
85975   1991-01-14
85976   1991-01-14
Name: tourney_date, dtype: datetime64[ns]


In [77]:
# i want to drop the match_num column 
df = df.drop(columns=['match_num'])
df = df.drop(columns=["winner_entry", "loser_entry"])  # drop the entry columns
df = df.drop(columns=["winner_ioc", "loser_ioc"])  # drop the ioc columns
display(df.head())  # check the first few rows of the DataFrame

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,winner_id,winner_seed,winner_name,winner_hand,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
85972,1991-580,australian open,Hard,128,G,1991-01-14,101222,1.0,Stefan Edberg,R,...,72.0,43.0,11.0,14.0,4.0,10.0,1.0,3889.0,162.0,202.0
85973,1991-580,australian open,Hard,128,G,1991-01-14,100995,NaN,Eduardo Masso,L,...,110.0,67.0,29.0,26.0,7.0,18.0,124.0,278.0,81.0,453.0
85974,1991-580,australian open,Hard,128,G,1991-01-14,101138,NaN,Pat Cash,R,...,59.0,31.0,12.0,12.0,9.0,16.0,84.0,432.0,32.0,845.0
85975,1991-580,australian open,Hard,128,G,1991-01-14,100870,NaN,Christo Van Rensburg,R,...,49.0,26.0,26.0,14.0,8.0,15.0,59.0,527.0,82.0,448.0
85976,1991-580,australian open,Hard,128,G,1991-01-14,101731,NaN,Nuno Marques,L,...,77.0,57.0,28.0,23.0,9.0,13.0,104.0,340.0,116.0,299.0
